In [1]:
# Okay, let's get the proper transformations, and also get all the predictions on a subwindow scale - Later as a proof of concept, we will separate the sequences with densities all zero and see how the predicted densities look like.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr

In [3]:
# path to dataframes
path_to_dfs = "data/BLAR_implementation/Block_0106/forecasted_counts"

In [4]:
len(os.listdir(path_to_dfs))

21504

In [5]:
len(os.listdir(path_to_dfs)) / 1376

7.0

In [6]:
averaged_forecasts_csvs = []
for file in os.listdir(path_to_dfs):
    if file[:8] =='averaged':
        averaged_forecasts_csvs.append(file)

In [7]:
# how many csv files do we have?
len(averaged_forecasts_csvs)

3072

In [8]:
averaged_forecasts_csvs[0]

'averaged_forecasts_sub_986.csv'

In [9]:
# let's create these names in the order of the subwindows
ordered_csv_files = ['averaged_forecasts_sub_' + str(i) + '.csv' for i in range(1376)]

In [10]:
# ordered_csv_files

In [ ]:
%%time
all_dfs = []
for file in ordered_csv_files:
    read_df = pd.read_csv(path_to_dfs + '/' + file)
    all_dfs.append(read_df)

In [ ]:
# make a single dataframe out of the many
combined_df = pd.concat(all_dfs, axis = 1)

In [ ]:
combined_df.head()

In [ ]:
combined_df.shape

In [ ]:
# get the true values
True_values_df = combined_df[['True_value']]

In [ ]:
True_values_df.head()

In [ ]:
True_values_df.shape

In [ ]:
# get the proper counts? - Do the transformation
exp_true = np.exp(True_values_df)-1

In [ ]:
exp_true

In [ ]:
# Identify the maximum and the minimum true values we have for the test data

In [ ]:
exp_true.values.min(), exp_true.values.max()

In [ ]:
# Okay, so the highest true value we have is ~3. let's get teh min and max values for the predicted exponentiated values also

In [ ]:
all_forecasts_npy_files = []
for file in os.listdir(path_to_dfs):
    if file[:13] == 'all_forecasts':
        all_forecasts_npy_files.append(file)

In [ ]:
len(all_forecasts_npy_files)

In [ ]:
all_forecasts_npy_files[0]

In [ ]:
# load just the first file
sub_273_forecasts = np.load(os.path.join(path_to_dfs, all_forecasts_npy_files[0]))

In [ ]:
sub_273_forecasts.shape

In [ ]:
# Okay, so for the subwindow 273 -  this file seem to have all the 1000 values in the 4 chains for all the test time points in the sequence

In [ ]:
# transform these values?
sub_273_forecasts = np.exp(sub_273_forecasts) - 1

In [ ]:
sub_273_forecasts.min(), sub_273_forecasts.max()

In [ ]:
# get rid of anything below zero?
sub_273_forecasts[sub_273_forecasts < 0] = 0

In [ ]:
sub_273_forecasts.min(), sub_273_forecasts.max()

In [ ]:
# cool -  what does these values mean?

In [ ]:
# try averaging the forecasted values along the 0th and 1st axis?

In [ ]:
sub_273_forecasts_averged = np.mean(sub_273_forecasts, axis = (0,1))

In [ ]:
sub_273_forecasts_averged

In [ ]:
sub_273_forecasts_averged.shape

In [ ]:
# sanity check

In [ ]:
# sanity check
for i in range(sub_273_forecasts.shape[-1]):
    track = sub_273_forecasts[:,:,i]
    print(np.mean(track))

In [ ]:
# Verify the shape of the true values - these are at a subwindow level, and we can use these as they are for getting the metrics, and the scatterplots?

In [ ]:
exp_true.shape

In [ ]:
# Okay, we need the predictions for the subwindows in the same shape? - but before that, we need to do the transformation for all the npy files. We can do this in a loop - we have done this before, use the code as it is.

In [ ]:
%%time
catch_all_averaged_preds = []
for i in range(exp_true.shape[1]):
    file_name = "all_forecasts_sub_" + str(i) + '.npy'
    al_forecasts_npy_file = np.load(os.path.join(path_to_dfs, file_name))
    # make the conversion
    sub_window_forecasts = np.exp(al_forecasts_npy_file) - 1
    # get rid of anything below zero?
    sub_window_forecasts[sub_window_forecasts < 0] = 0
    # get averages over time 
    sub_window_forecasts_averaged = np.mean(sub_window_forecasts, axis = (0,1))
    catch_all_averaged_preds.append(sub_window_forecasts_averaged)    

In [ ]:
len(catch_all_averaged_preds)

In [ ]:
catch_all_averaged_preds[0]

In [ ]:
Forecasted_values_df = pd.DataFrame(catch_all_averaged_preds).T

In [ ]:
Forecasted_values_df.shape

In [ ]:
Forecasted_values_df.head()

In [ ]:
# give column names here
Forecasted_values_df.columns = ['forecasted_val_' + str(i) for i in range(1376)]

In [ ]:
Forecasted_values_df.head()

In [ ]:
# get the min and max values here?

In [ ]:
Forecasted_values_df.values.min(), Forecasted_values_df.values.max()

In [59]:
# notice that, there are a lot of values which seem to be greater than 20 - this seems like an okay value to have a cut off - should we try to get the row and column indices of these values? Maybe we need to run these later individually until we get stable values?

In [60]:
mask_gt_20 = Forecasted_values_df > 20

In [61]:
resulting_df = Forecasted_values_df[mask_gt_20].stack().reset_index()

In [62]:
resulting_df.head()

,level_0,level_1,0
0,3,forecasted_val_6,73.062149
1,3,forecasted_val_7,423.084747
2,3,forecasted_val_8,58.340443
3,3,forecasted_val_9,39.000557
4,3,forecasted_val_27,38.795723


In [63]:
resulting_df.shape

(1166, 3)

In [64]:
resulting_df.columns = ['row_index', 'column_name', 'forecasted_val']

In [65]:
resulting_df.head()

,row_index,column_name,forecasted_val
0,3,forecasted_val_6,73.062149
1,3,forecasted_val_7,423.084747
2,3,forecasted_val_8,58.340443
3,3,forecasted_val_9,39.000557
4,3,forecasted_val_27,38.795723


In [66]:
# how many coumns do we need to focus on?
resulting_df['column_name'].value_counts()

column_name
forecasted_val_1144    4
forecasted_val_1145    4
forecasted_val_1149    4
forecasted_val_1185    4
forecasted_val_1186    4
                      ..
forecasted_val_666     1
forecasted_val_1064    1
forecasted_val_1053    1
forecasted_val_284     1
forecasted_val_1103    1
Name: count, Length: 340, dtype: int64

In [67]:
# notice that there are 347 py scripts that we may need to rerun

In [68]:
# What are the subwindow names?

In [69]:
sub_window_names = list(set(resulting_df['column_name']))
sub_window_names.sort()

In [70]:
# sub_window_names - but this is not properly sorted

In [71]:
sub_window_numbers = [val.split('_')[-1] for val in resulting_df['column_name']]
sub_window_int = [int(i) for i in sub_window_numbers]
sub_window_int.sort()

In [72]:
len(sub_window_int)

1166

In [73]:
unique_subwindow_int = list(set(sub_window_int))

In [74]:
len(unique_subwindow_int)

340

In [75]:
sub_window_names_final = ['forecasted_val_' + str(i) for i in unique_subwindow_int]

In [76]:
# sub_window_names_final # cool, so we need to look into individually running these scripts?

In [77]:
# We are waiting to see if changing the prioirs are going to give us a direction we should proceed in

In [ ]:
# The maximmum is an absurd amount - where is this coming from ? Do we need to manually fix it? Let's see
# how many of teh values in the above dataframe are more than 10?

In [ ]:
# Maybe see to this once all columns are arranged in a single column?

In [ ]:
exp_true.columns = ['True_val_' + str(i) for i in range(1376)]

In [ ]:
exp_true.head()

In [ ]:
# I think at this point we can go ahead and get the metrics for inference computed?

In [ ]:
# exp_true.keys()

In [ ]:
# exp_true.values.shape

In [ ]:
# use one column for both dataframes? that way it will be easier to compute the metrics?

In [ ]:
# Also something to keep in mind here is that the subwindow sizes are very small - 30*30 - this was intentional as we had to do the seq-2-seq model and they are data hungry - therefore having a 300*300 window size was not feasible

In [ ]:
exp_true_onecol = pd.DataFrame(exp_true.to_numpy().ravel(order = 'F'), columns=["True_values"])

In [ ]:
exp_true_onecol.head()

In [ ]:
exp_true_onecol.shape

In [ ]:
3072*7

In [ ]:
exp_predicted_onecol = pd.DataFrame(Forecasted_values_df.to_numpy().ravel(order = 'F'), columns=["Forecasted_values"])

In [ ]:
exp_predicted_onecol.head()

In [ ]:
exp_predicted_onecol.tail()

In [ ]:
exp_predicted_onecol.shape

In [ ]:
# how many values are > 10? - 298 - This still feels like an absolutely high value. Should we change the priors? Need to look into this. 
# how many values are > 100? - 82
# how many values are > 1000? - 27
# how many values are > 10000? - 10
# how many values are > 100000? - 0

In [ ]:
values_greater_than_10 = []
greater_than_10_df_index = []
for i, val in enumerate(range(exp_predicted_onecol.shape[0])):
    value = exp_predicted_onecol['Forecasted_values'][val]
    if value >= 10:
        values_greater_than_10.append(value)
        greater_than_10_df_index.append(i)

In [ ]:
len(values_greater_than_10)

In [ ]:
# values_greater_than_10

In [ ]:
# greater_than_10_df_index

In [ ]:
# enumerate(exp_predicted_onecol['Forecasted_values'][0])

In [ ]:
# I think at this point, we should be looking inot other blocks too, to see if we see weird results like this?

In [ ]:
# compute the metrics - these would look really small as the values we have are really small

In [ ]:
# blockwise mean squared error
rmse = np.sqrt(mean_squared_error(exp_true_onecol['True_values'], exp_predicted_onecol['Forecasted_values']))
rmse

In [ ]:
mae = mean_absolute_error(exp_true_onecol['True_values'], exp_predicted_onecol['Forecasted_values'])
mae

In [ ]:
corr = pearsonr(exp_true_onecol['True_values'], exp_predicted_onecol['Forecasted_values'])
corr[0]

In [ ]:
r2 = r2_score(exp_true_onecol['True_values'], exp_predicted_onecol['Forecasted_values'])
r2

In [ ]:
plt.figure(figsize = (8,8))
plt.scatter(exp_true_onecol['True_values'], exp_predicted_onecol['Forecasted_values'], s = 10)
plt.xlabel("True density")
plt.ylabel("Predicted density")
# Add the y = x line
plt.plot([0, 3], [0, 3], color='green', label='y = x')
plt.legend()
plt.show()

In [ ]:
# At this point let's look at the true and predicted value distributions - overlay the histograms

# Plot histogram of two columns
plt.hist(exp_true_onecol['True_values'], 
         bins=30, 
         label=[' True values'], 
         alpha=0.5, color = 'blue')  # alpha = transparency
plt.hist(exp_predicted_onecol['Forecasted_values'], 
         bins=30, 
         label=['Predicted values'], 
         alpha=0.5, color = 'red') 
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of Two Columns")
plt.legend()
plt.show()

In [ ]:
# Should we try to get the values separated by 0s?

In [ ]:
# how to do this? Maybe we need to take the indices from the true values df, and subset accordingly?

In [ ]:
# so create a mask 0 with the true values 0s - we need the indices of this

In [ ]:
mask_zero = exp_true_onecol['True_values'].eq(0)

In [ ]:
# separate the zero and non-zero indices
idx_zero = exp_true_onecol.index[mask_zero]
idx_nonzero = exp_true_onecol.index[-mask_zero]

In [ ]:
# how many zero values?
idx_zero.shape

In [ ]:
910*7

In [ ]:
# how many non-zeor values
idx_nonzero.shape

In [ ]:
# sanity check
idx_nonzero.shape[0] + idx_zero.shape[0]

In [ ]:
# Subset both dataframes with the same indices
df_true_zero  = exp_true_onecol.loc[idx_zero]
df_pred_zero  = exp_predicted_onecol.loc[idx_zero]
df_true_nz    = exp_true_onecol.loc[idx_nonzero]
df_pred_nz    = exp_predicted_onecol.loc[idx_nonzero]

In [ ]:
df_true_zero.shape, df_pred_zero.shape

In [ ]:
df_true_zero.head()

In [ ]:
df_pred_zero.head()

In [ ]:
df_true_nz.head()

In [ ]:
df_pred_nz.head()

In [ ]:
# create the separate scatterplots for these

In [ ]:
# For zero-data
plt.figure(figsize = (8,8))
plt.scatter(df_true_zero['True_values'], df_pred_zero['Forecasted_values'], s = 10)
plt.title("Scatter plot when the True densities are zeros")
plt.xlabel("True densities")
plt.ylabel("Forecasted densities")
plt.show()

In [ ]:
# Plot histogram of two columns
plt.hist(df_true_zero['True_values'], 
         bins=30, 
         label=[' True values'], 
         alpha=0.5, color = 'blue')  # alpha = transparency
plt.hist(df_pred_zero['Forecasted_values'], 
         bins=30, 
         label=['Predicted values'], 
         alpha=0.5, color = 'red') 
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of Two Columns")
plt.legend()
plt.show()

In [ ]:
# For non-zero data
plt.figure(figsize = (8,8))
plt.scatter(df_true_nz['True_values'], df_pred_nz['Forecasted_values'], s = 10)
plt.title("Scatter plot when the True densities are non-zeros")
# also plot the y = x line?
plt.plot([0, 3], [0, 3], color='green', label='y = x')
plt.xlabel("True densities")
plt.ylabel("Forecasted densities")
plt.show()

In [ ]:
# Plot histogram of two columns
plt.hist(df_true_nz['True_values'], 
         bins=30, 
         label=[' True values'], 
         alpha=0.5, color = 'blue')  # alpha = transparency
plt.hist(df_pred_nz['Forecasted_values'], 
         bins=30, 
         label=['Predicted values'], 
         alpha=0.5, color = 'red') 
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.title("Histogram of Two Columns")
plt.legend()
plt.show()

In [ ]:
# not that great - but maybe the model is not doing as well as we need due to the number of zeros we had considered during the model training phase?

In [ ]:
# Maybe get the metrics for teh reduced subsets of data as well

In [ ]:
# All zeros

In [ ]:
rmse_zero =  np.sqrt(mean_squared_error(df_true_zero['True_values'], df_pred_zero['Forecasted_values']))
rmse_zero

In [ ]:
mae_zero = mean_absolute_error(df_true_zero['True_values'], df_pred_zero['Forecasted_values'])
mae_zero

In [ ]:
corr_zero = pearsonr(df_true_zero['True_values'], df_pred_zero['Forecasted_values'])

In [ ]:
corr_zero

In [ ]:
r2_zero = r2_score(df_true_nz['True_values'], df_pred_nz['Forecasted_values'])
r2_zero

In [ ]:
# non zeros

In [ ]:
rmse_nzero =  np.sqrt(mean_squared_error(df_true_nz['True_values'], df_pred_nz['Forecasted_values']))
rmse_nzero

In [ ]:
mae_nzero = mean_absolute_error(df_true_nz['True_values'], df_pred_nz['Forecasted_values'])
mae_nzero

In [ ]:
corr_nzero = pearsonr(df_true_nz['True_values'], df_pred_nz['Forecasted_values'])

In [ ]:
corr_nzero

In [ ]:
r2_nzero = r2_score(df_true_nz['True_values'], df_pred_nz['Forecasted_values'])
r2_nzero

Credible Intervals

In [ ]:
all_forecasts_npy_files = ['all_forecasts_sub_' + str(i) + '.npy' for i in range(1376)]

In [ ]:
%%time
loaded_npy_files = []
for file in all_forecasts_npy_files:
    joined_path = os.path.join(path_to_dfs, file)
    load_file = np.load(joined_path)
    # notice we are averaging the preds across the chains before exponentiation
    mean_file = np.mean(np.exp(load_file)-1, axis = 1)
    loaded_npy_files.append(mean_file)

In [ ]:
len(loaded_npy_files)

In [ ]:
loaded_npy_files[0].shape

In [ ]:
# Maybe we should not get rid of the negatives? - As for the percentiles we will else never capture the true values of zeros?

In [ ]:
# yeah, let's proceed with these

In [ ]:
# how to arrange these though? - might need to think this through a little

In [ ]:
exp_true.shape

In [ ]:
# where is the lsit of the forecasted range of values? - loaded_npy_files

In [ ]:
len(loaded_npy_files)

In [ ]:
loaded_npy_files[0].shape

In [ ]:
# Okay, I think we need to move along the loaded file and also its axis 1 to get the percentile values in order.

In [ ]:
# create a for loop for this? - and later maybe even a function so we do not need to repeat everything for each block separately

In [ ]:
# I think we also need to catch the percentiles?

In [ ]:
# Let's try all these?

In [ ]:
trial_0 = loaded_npy_files[0][:,0]

In [ ]:
trial_0.shape

In [ ]:
trial_li = np.percentile(trial_0, axis = 0, q = (2.5, 97.5))

In [ ]:
# trial_0.min(), trial_0.max()

In [ ]:
trial_li

In [ ]:
trial_li[0], trial_li[1]

In [ ]:
lower_and_upper_limits = [] 
for j in range(loaded_npy_files[0].shape[1]):
    values = loaded_npy_files[0][:,j]
    # compute the lower and upper bounds?
    li = np.percentile(values, axis = 0, q = (2.5, 97.5))[0]    
    ui = np.percentile(values, axis = 0, q = (2.5, 97.5))[1]
    lower_and_upper_limits.append((li, ui))

In [ ]:
lower_and_upper_limits

In [ ]:
%%time
# Do this for all files?
catch_all_percentiles = []
for i in range(len(loaded_npy_files)):
    all_lower_and_upper_limits = [] 
    for j in range(loaded_npy_files[i].shape[1]):
        values = loaded_npy_files[i][:,j]
        # compute the lower and upper bounds?
        li = np.percentile(values, axis = 0, q = (2.5, 97.5))[0]    
        ui = np.percentile(values, axis = 0, q = (2.5, 97.5))[1]
        all_lower_and_upper_limits.append((li, ui))
    catch_all_percentiles.append(all_lower_and_upper_limits)
    

In [ ]:
len(catch_all_percentiles)

In [ ]:
catch_inside_length = []
for limit_values in catch_all_percentiles:
    length = len(limit_values)
    catch_inside_length.append(length)

In [ ]:
np.mean(catch_inside_length)

In [ ]:
# We may need a multitude of sanity checks to make sure we are correctly computing the coverages and widths. Now at this point, we can go ahead and compute the CI widths using the credible intervals? As this computation seems fairly simple?

In [ ]:
len(catch_all_percentiles[0])

In [ ]:
all_percentiles_for_widths = [item for limit_values in catch_all_percentiles for item in limit_values]

In [ ]:
len(all_percentiles_for_widths)

In [ ]:
# do some sanity check?

In [ ]:
catch_all_percentiles[1]

In [ ]:
all_percentiles_for_widths[7:14]

In [ ]:
# seems this is right, let's move forward with the computation of the CI widths?

In [ ]:
catch_wdths = []
for values in all_percentiles_for_widths:
    width = values[1] - values[0]
    catch_wdths.append(width)

In [ ]:
# average CI width
Average_CI_width = np.mean(catch_wdths)
Average_CI_width

In [ ]:
# do a few sanity checks  before we proceed?

In [ ]:
catch_wdths[0:7]

In [ ]:
print(catch_all_percentiles[0][0][1] - catch_all_percentiles[0][0][0])
print(catch_all_percentiles[0][1][1] - catch_all_percentiles[0][1][0])
print(catch_all_percentiles[0][2][1] - catch_all_percentiles[0][2][0])
print(catch_all_percentiles[0][3][1] - catch_all_percentiles[0][3][0])
print(catch_all_percentiles[0][4][1] - catch_all_percentiles[0][4][0])
print(catch_all_percentiles[0][5][1] - catch_all_percentiles[0][5][0])
print(catch_all_percentiles[0][6][1] - catch_all_percentiles[0][6][0])

In [ ]:
# Okay, we can move on now

In [ ]:
# What about the coverage?
# Now this list - all_percentiles_for_widths - this is in the oder of the subwindows - but before proceeding to the next subwindow, it also tracks across the 7 time periods.
# so now, we have the true values in a dataframe in the shape (7,910), we can stack these ina single column  - and basically then have to track if this value is inbetween the two upper and lower limits in the list all_percentiles_for_widths.

In [ ]:
# cool, so let's get this true values dataset arranged  in one single column?

In [ ]:
# actually, we have already done that

In [ ]:
exp_true_onecol.shape

In [ ]:
exp_true_onecol.head()

In [ ]:
All_exp_true_vals_array = exp_true_onecol.values.reshape(-1)

In [ ]:
All_exp_true_vals_array.shape

In [ ]:
All_exp_true_vals_array.min(), All_exp_true_vals_array.max()

In [ ]:
# cool, now check if this value is inbetween the upper and the lower limits contained in the list - all_percentiles_for_widths

In [ ]:
Catch_all_indicators = []
for i in range(All_exp_true_vals_array.shape[0]):
    true_value = All_exp_true_vals_array[i]
    li_val = all_percentiles_for_widths[i][0]
    ui_val = all_percentiles_for_widths[i][1]
    ind_train = (true_value >= li_val) & (true_value <= ui_val)
    Catch_all_indicators.append(ind_train)

In [ ]:
coverage_val = np.mean(Catch_all_indicators)
coverage_val

In [ ]:
# I think we are ready to replicate this for the rest of the blocks? Let's push the recent work to GitHub

In [ ]:
# define a function to get the post-hoc prediction
def prediction_on_test_data(preds_array, stride = 24, kernel_size = 300):

    img_height = 768 # we might need to verify this value
    # get the image weight
    img_width = 1024 # we might need to verify this value

    
    # create an empty density map
    Density_map = np.zeros((img_height, img_width))

    # create an empty counts map
    counts_map = np.zeros((img_height, img_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, img_height, stride):
        for jj in range(0, img_width, stride):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), preds_array[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)
    
#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts)

In [ ]:
Forecasted_values_df.shape

In [ ]:
Forecasted_values_df.values.shape

In [ ]:
# Access just the first row?

In [ ]:
Forecasted_values_df.values[0,:].shape

In [ ]:
# use this to get the preds?

In [ ]:
test_im_0_preds = prediction_on_test_data(Forecasted_values_df.values[0,:], stride = 24, kernel_size = 300)

In [ ]:
test_im_0_preds[0]

In [ ]:
# get the values forecasted in a loop?
predicted_test_values = []
for i in range(7):
    pred_val = prediction_on_test_data(Forecasted_values_df.values[i,:], stride = 24, kernel_size = 300)[0]
    predicted_test_values.append(pred_val)

In [ ]:
predicted_test_values

In [ ]:
# Okay, there's some seroius shit happening I think with those very high values - we need to figuer out what to do